In [1]:
#clear all
%reset -f

#import packages
import numpy as np
import scipy
import sys
import os
import pandas as pd
import mne
import matplotlib
import h5py
from sklearn.utils import resample
from mne_icalabel import label_components

root = 'F:/Documents/Science/MirRevAdaptEEG'
participants = list(range(0,32))
#specify which erp we are analyzing
erps = 'lrp'
roi = 'medfro'

#pop up plots as separate window & interactive
%matplotlib qt
matplotlib.pyplot.close('all')

In [2]:
#setting up path/ directory
#access trial type epoched data for each participant
def load_tfr_epochs(pp_num, root_dir, erp_path, task):

    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    filename = os.path.join(pp_directory, 'p%03d_%s-epo.fif' % (pp_num, task))

    epochs = mne.read_epochs(filename)

    return epochs, pp_directory

In [3]:
#set up parameters for time-frequency morlet convolution
def set_tfr_params(l_freq = 6, h_freq = 35, num_freq = 50, cycle = 6, samp_rate = 200):
    freqs = np.logspace(*np.log10([l_freq, h_freq]), num = num_freq)
    n_cycles = cycle
    sfreq = samp_rate
    
    return freqs, n_cycles, sfreq

In [4]:
#save tfr'd data
def save_tfr_data(pp_num, root_dir, erp_path, data, task, output):
    # Save the tfr'd data
    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    out_fname = os.path.join(pp_directory, 'p%03d_%s_%s-tfr.h5' % (pp_num, task, output))
    mne.time_frequency.write_tfrs(out_fname, tfr=data, overwrite = True)

In [5]:
#load tfr'd data
def load_tfr_data(pp_num, root_dir, erp_path, task, output):
    # Save the tfr'd data
    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    out_fname = os.path.join(pp_directory, 'p%03d_%s_%s-tfr.h5' % (pp_num, task, output))
    dat = mne.time_frequency.read_tfrs(out_fname)
    
    return dat

In [6]:
# After transforming the TFR data into a single signal representative of frequency and channels
# we compare the two signals: same as taking their difference (signal1 minus signal2) and using this in permutation test
def get_clust_perm_test(conditionA, conditionB, pval, n_permutations):
    #take difference of two conditions
    data = np.subtract(conditionA, conditionB)
    np.shape(data)
    #define cluster forming threshold based on p-value
    df = len(participants) - 1
    thresh = scipy.stats.t.ppf(1 - pval / 2, df)
    #run cluster-based permutation test
    T_0, clust_idx, clust_pvals, H0 = mne.stats.permutation_cluster_1samp_test(data, threshold = thresh, 
                                                          n_permutations = n_permutations, tail = 0, 
                                                          adjacency = None, seed = 999, 
                                                          out_type = 'mask', verbose = True)

    return T_0, clust_idx, clust_pvals, H0

In [7]:
# # save average tfr per participant for each condition
# # morlet wavelet convolution for each participant, according to trial types - POWER

# perturb_conds = ['early_late_aligned', 'early_rot', 'late_rot', 'early_mir', 'late_mir', 'early_rdm', 'late_rdm']

# for pcond in range(0, len(perturb_conds)):
#     for pp in participants:
#         data, ppdir = load_tfr_epochs(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond])
#         #use all trials for aligned
#         if pcond == 1:
#             data = data[0:12] #first 12/45 trials for early
#         elif pcond == 2:
#             data = data[33:45] #last 12/45 trials for late
#         elif pcond == 3:
#             data = data[0:12]
#         elif pcond == 4:
#             data = data[33:45]
#         elif pcond == 5:
#             data = data[0:12]
#         elif pcond == 6:
#             data = data[36:48] #rdm has 48 trials only
        
#         freqs, n_cycles, sfreq = set_tfr_params()
        
#         #average epochs within participant
#         power = mne.time_frequency.tfr_morlet(
#             data,
#             freqs=freqs,
#             n_cycles=n_cycles,
#             use_fft=False,
#             decim=1,
#             n_jobs=None,
#             return_itc = False,
#             average = True
#         )
        
#         save_tfr_data(pp_num = pp, root_dir = root, erp_path = erps, data = power, task = perturb_conds[pcond], output = "power")

In [8]:
# load in tfr object, ensure to baseline correct; (tfr shape is channels, freqs, timepts)
# narrow down to channels 
# narrow down to frequencies and timepts
# take mean of alpha or beta ranges (freqs)
# take mean across electrodes (channels)

perturb_conds = ['early_late_aligned', 'early_rot', 'late_rot', 'early_mir', 'late_mir', 'early_rdm', 'late_rdm']

# medial frontal CHANNELS
channels = ['F1', 'Fz', 'F2',
            'FC1', 'FCz', 'FC2',
            'C1', 'Cz', 'C2']


# BASELINE FOR GO SIGNAL ONSET
baseline_t = (-1.3, -1.0)

# ALPHA
freq_lower = 9
freq_upper = 13

alpha_medfro_EarlyLateAligned = []
alpha_medfro_EarlyRot = []
alpha_medfro_LateRot = []
alpha_medfro_EarlyMir = []
alpha_medfro_LateMir = []
alpha_medfro_EarlyRdm = []
alpha_medfro_LateRdm = []

for pcond in range(0, len(perturb_conds)):
    medfroppdat = []
    for pp in participants:
        dat = load_tfr_data(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond], output = "power")
        dat = dat[0]*1e12 #convert V^2 to uV^2
        # apply baseline correction
        dat = dat.apply_baseline(baseline_t, mode='mean')
        # narrow down to channels
        dat = dat.pick_channels(channels)
        #transform to ndarray (channels, freqs, times)
        npdat = dat.data

        # get needed frequency indices
        nfreqs = [] #get indices of frequencies we want
        for i in range(0, len(dat.freqs)):
            if dat.freqs[i] >= freq_lower and dat.freqs[i] <= freq_upper:
                nfreqs.append(i)
        npdat = npdat[:, nfreqs, :]

        # get needed timept indices
        ntimes = list(range(100,401))
        npdat = npdat[:,:,ntimes]
        
        ppdat = np.mean(npdat, axis = 1) #calculate mean across frequencies
        ppdat = np.mean(ppdat, axis = 0) # calculate mean across channels
        
        medfroppdat.append(ppdat)
        
    if pcond == 0:
        alpha_medfro_EarlyLateAligned.append(medfroppdat)
    elif pcond == 1:
        alpha_medfro_EarlyRot.append(medfroppdat)
    elif pcond == 2:
        alpha_medfro_LateRot.append(medfroppdat)
    elif pcond == 3:
        alpha_medfro_EarlyMir.append(medfroppdat)
    elif pcond == 4:
        alpha_medfro_LateMir.append(medfroppdat)
    elif pcond == 5:
        alpha_medfro_EarlyRdm.append(medfroppdat)
    elif pcond == 6:
        alpha_medfro_LateRdm.append(medfroppdat)

# BETA
freq_lower = 13
freq_upper = 25

beta_medfro_EarlyLateAligned = []
beta_medfro_EarlyRot = []
beta_medfro_LateRot = []
beta_medfro_EarlyMir = []
beta_medfro_LateMir = []
beta_medfro_EarlyRdm = []
beta_medfro_LateRdm = []

for pcond in range(0, len(perturb_conds)):
    medfroppdat = []
    for pp in participants:
        dat = load_tfr_data(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond], output = "power")
        dat = dat[0]*1e12 #convert V^2 to uV^2
        # apply baseline correction
        dat = dat.apply_baseline(baseline_t, mode='mean')
        # narrow down to channels
        dat = dat.pick_channels(channels)
        #transform to ndarray (channels, freqs, times)
        npdat = dat.data

        # get needed frequency indices
        nfreqs = [] #get indices of frequencies we want
        for i in range(0, len(dat.freqs)):
            if dat.freqs[i] >= freq_lower and dat.freqs[i] <= freq_upper:
                nfreqs.append(i)
        npdat = npdat[:, nfreqs, :]

        # get needed timept indices
        ntimes = list(range(100,401))
        npdat = npdat[:,:,ntimes]
        
        ppdat = np.mean(npdat, axis = 1) #calculate mean across frequencies
        ppdat = np.mean(ppdat, axis = 0) # calculate mean across channels
        
        medfroppdat.append(ppdat)
        
    if pcond == 0:
        beta_medfro_EarlyLateAligned.append(medfroppdat)
    elif pcond == 1:
        beta_medfro_EarlyRot.append(medfroppdat)
    elif pcond == 2:
        beta_medfro_LateRot.append(medfroppdat)
    elif pcond == 3:
        beta_medfro_EarlyMir.append(medfroppdat)
    elif pcond == 4:
        beta_medfro_LateMir.append(medfroppdat)
    elif pcond == 5:
        beta_medfro_EarlyRdm.append(medfroppdat)
    elif pcond == 6:
        beta_medfro_LateRdm.append(medfroppdat)
    

Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p000/lrp\p000_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p001/lrp\p001_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p002/lrp\p002_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p003/lrp\p003_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p004/lrp\p004_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p005/lrp\p005_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p006/lrp\p006_early_late_aligned_power-tfr.h5 ...
Applying baseline correctio

Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p027/lrp\p027_early_rot_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p028/lrp\p028_early_rot_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p029/lrp\p029_early_rot_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p030/lrp\p030_early_rot_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p031/lrp\p031_early_rot_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p000/lrp\p000_late_rot_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p001/lrp\p001_late_rot_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading 

Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p024/lrp\p024_early_mir_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p025/lrp\p025_early_mir_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p026/lrp\p026_early_mir_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p027/lrp\p027_early_mir_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p028/lrp\p028_early_mir_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p029/lrp\p029_early_mir_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p030/lrp\p030_early_mir_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Readin

Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p021/lrp\p021_early_rdm_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p022/lrp\p022_early_rdm_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p023/lrp\p023_early_rdm_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p024/lrp\p024_early_rdm_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p025/lrp\p025_early_rdm_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p026/lrp\p026_early_rdm_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p027/lrp\p027_early_rdm_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Readin

Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p017/lrp\p017_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p018/lrp\p018_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p019/lrp\p019_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p020/lrp\p020_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p021/lrp\p021_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p022/lrp\p022_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p023/lrp\p023_early_late_aligned_po

Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p013/lrp\p013_late_rot_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p014/lrp\p014_late_rot_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p015/lrp\p015_late_rot_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p016/lrp\p016_late_rot_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p017/lrp\p017_late_rot_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p018/lrp\p018_late_rot_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p019/lrp\p019_late_rot_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Do

Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p010/lrp\p010_late_mir_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p011/lrp\p011_late_mir_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p012/lrp\p012_late_mir_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p013/lrp\p013_late_mir_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p014/lrp\p014_late_mir_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p015/lrp\p015_late_mir_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p016/lrp\p016_late_mir_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Do

Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p007/lrp\p007_late_rdm_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p008/lrp\p008_late_rdm_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p009/lrp\p009_late_rdm_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p010/lrp\p010_late_rdm_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p011/lrp\p011_late_rdm_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p012/lrp\p012_late_rdm_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p013/lrp\p013_late_rdm_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Do

In [9]:
# Get timepts to use for indices given by cluster-based permutation later and for timepts in saved data frames
# But only grab -0.25 to 1.5 sec time-locked to feedback onset (idx= 350:701)
root_directory = root
pp = 0 #only need one participant

# we can use aligned data
dat = load_tfr_data(pp_num = pp, root_dir = root, erp_path = erps, task = 'early_late_aligned', output = "power")
time = dat[0].times
time = time[100:401] #get only timepoints we want

Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p000/lrp\p000_early_late_aligned_power-tfr.h5 ...


In [10]:
# save tfr data across participants in each condition (used for plotting signals)
alpha_flists = [alpha_medfro_EarlyLateAligned, alpha_medfro_EarlyRot, alpha_medfro_LateRot, 
                alpha_medfro_EarlyRdm, alpha_medfro_LateRdm, 
                alpha_medfro_EarlyMir, alpha_medfro_LateMir]
beta_flists = [beta_medfro_EarlyLateAligned, beta_medfro_EarlyRot, beta_medfro_LateRot, 
               beta_medfro_EarlyRdm, beta_medfro_LateRdm,
               beta_medfro_EarlyMir, beta_medfro_LateMir]

alpha_conditionnames = ['alpha_earlylate_aligned', 'alpha_earlyrot', 'alpha_laterot', 'alpha_earlyrdm', 'alpha_laterdm', 'alpha_earlymir', 'alpha_latemir']
beta_conditionnames = ['beta_earlylate_aligned', 'beta_earlyrot', 'beta_laterot', 'beta_earlyrdm', 'beta_laterdm', 'beta_earlymir', 'beta_latemir']

freq_bands = ['alpha', 'beta']

for band in range(0, len(freq_bands)):
    if band == 0:
        for f in range(0, len(alpha_flists)):
            df = pd.DataFrame(alpha_flists[f][0]).T
            df['time'] = time

            filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_medfro_%s_%s.csv' % (alpha_conditionnames[f], erps))
            df.to_csv(filename)
    elif band == 1:
        for f in range(0, len(beta_flists)):
            df = pd.DataFrame(beta_flists[f][0]).T
            df['time'] = time

            filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_medfro_%s_%s.csv' % (beta_conditionnames[f], erps))
            df.to_csv(filename)
            

In [11]:
# First, we can compare each condition to aligned
# Generate a data frame to tabulate condition, cluster indices, cluster timepts, p values
# This information can then be included in plots

alpha_flists = [alpha_medfro_EarlyRot, alpha_medfro_LateRot, 
                alpha_medfro_EarlyRdm, alpha_medfro_LateRdm, 
                alpha_medfro_EarlyMir, alpha_medfro_LateMir]
beta_flists = [beta_medfro_EarlyRot, beta_medfro_LateRot, 
               beta_medfro_EarlyRdm, beta_medfro_LateRdm,
               beta_medfro_EarlyMir, beta_medfro_LateMir]

alpha_conditionnames = ['alpha_earlyrot', 'alpha_laterot', 'alpha_earlyrdm', 'alpha_laterdm', 'alpha_earlymir', 'alpha_latemir']
beta_conditionnames = ['beta_earlyrot', 'beta_laterot', 'beta_earlyrdm', 'beta_laterdm', 'beta_earlymir', 'beta_latemir']

freq_bands = ['alpha', 'beta']
p = 0.05
perms = 1000

condition = []
clust_idx_start = []
clust_idx_end = []
time_start = []
time_end = []
p_values = []

for band in range(0, len(freq_bands)):
    if band == 0:
        for f in range(0, len(alpha_flists)):
            T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(alpha_flists[f][0], alpha_medfro_EarlyLateAligned[0], p, perms)
#             print(clust_idx, clust_pvals)
            if len(clust_idx) == 0:
                condition.append(alpha_conditionnames[f])
                clust_idx_start.append(np.nan)
                clust_idx_end.append(np.nan)
                time_start.append(np.nan)
                time_end.append(np.nan)
                p_values.append(np.nan)
            else:
                for clust in range(0, len(clust_idx)):
                    cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                    cluster_start = cluster.start
                    clust_idx_start.append(cluster_start)
        
                    cluster_end = cluster.stop
                    clust_idx_end.append(cluster_end)
        
                    time_idx_start = time[cluster_start]
                    time_start.append(time_idx_start)
        
                    time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                    time_end.append(time_idx_end)
        
                    clust_p = clust_pvals[clust]
                    p_values.append(clust_p)
        
                    condition.append(alpha_conditionnames[f])
            
    elif band == 1:
        for f in range(0, len(beta_flists)):
            T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(beta_flists[f][0], beta_medfro_EarlyLateAligned[0], p, perms)
#             print(clust_idx, clust_pvals)
            if len(clust_idx) == 0:
                condition.append(beta_conditionnames[f])
                clust_idx_start.append(np.nan)
                clust_idx_end.append(np.nan)
                time_start.append(np.nan)
                time_end.append(np.nan)
                p_values.append(np.nan)
            else:
                for clust in range(0, len(clust_idx)):
                    cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                    cluster_start = cluster.start
                    clust_idx_start.append(cluster_start)
        
                    cluster_end = cluster.stop
                    clust_idx_end.append(cluster_end)
        
                    time_idx_start = time[cluster_start]
                    time_start.append(time_idx_start)
        
                    time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                    time_end.append(time_idx_end)
        
                    clust_p = clust_pvals[clust]
                    p_values.append(clust_p)
        
                    condition.append(beta_conditionnames[f])
        
perm_test = pd.DataFrame(
    {'condition': condition,
        'clust_idx_start': clust_idx_start,
        'clust_idx_end': clust_idx_end,
        'time_start': time_start,
        'time_end': time_end,
        'p_values': p_values})

perm_test_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_Permutation_test_vsAligned_%s_%s.csv' % (erps, roi))
perm_test.to_csv(perm_test_filename)


stat_fun(H1): min=-1.703439 max=1.906422
Running initial clustering …
Found 0 clusters
stat_fun(H1): min=-2.258247 max=2.532507
Running initial clustering …
Found 2 clusters


C:\Users\Raphael\AppData\Local\Temp\ipykernel_2160\3082407755.py:11: RuntimeWarning: No clusters found, returning empty H0, clusters, and cluster_pv
  T_0, clust_idx, clust_pvals, H0 = mne.stats.permutation_cluster_1samp_test(data, threshold = thresh,


  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]

stat_fun(H1): min=-1.407888 max=3.469760
Running initial clustering …
Found 1 cluster


  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]

...

  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]

stat_fun(H1): min=-2.512615 max=1.543906
Running initial clustering …
Found 1 cluster


  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]

stat_fun(H1): min=-3.078969 max=1.925253
Running initial clustering …
Found 3 clusters


  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]

In [12]:
# Next, we subtract aligned from each condition, so that we can compare early vs late in each perturbation type

freq_bands = ['alpha', 'beta']
diffconds = ['earlyrot', 'laterot', 'earlyrdm', 'laterdm', 'earlymir', 'latemir']

alpha_medfro_earlyrot_diff = []
alpha_medfro_laterot_diff = []
alpha_medfro_earlyrdm_diff = []
alpha_medfro_laterdm_diff = []
alpha_medfro_earlymir_diff = []
alpha_medfro_latemir_diff = []

beta_medfro_earlyrot_diff = []
beta_medfro_laterot_diff = []
beta_medfro_earlyrdm_diff = []
beta_medfro_laterdm_diff = []
beta_medfro_earlymir_diff = []
beta_medfro_latemir_diff = []

for band in range(0, len(freq_bands)):
    if band == 0:
        for cond in range(0, len(diffconds)):
            if cond == 0:
                diffevks = np.subtract(alpha_medfro_EarlyRot[0], alpha_medfro_EarlyLateAligned[0])
                alpha_medfro_earlyrot_diff.append(diffevks)
                alpha_medfro_earlyrot_diff = alpha_medfro_earlyrot_diff[0] #to keep shape of object consistent
            elif cond == 1:
                diffevks = np.subtract(alpha_medfro_LateRot[0], alpha_medfro_EarlyLateAligned[0])
                alpha_medfro_laterot_diff.append(diffevks)
                alpha_medfro_laterot_diff = alpha_medfro_laterot_diff[0]
            elif cond == 2:
                diffevks = np.subtract(alpha_medfro_EarlyRdm[0], alpha_medfro_EarlyLateAligned[0])
                alpha_medfro_earlyrdm_diff.append(diffevks)
                alpha_medfro_earlyrdm_diff = alpha_medfro_earlyrdm_diff[0]
            elif cond == 3:
                diffevks = np.subtract(alpha_medfro_LateRdm[0], alpha_medfro_EarlyLateAligned[0])
                alpha_medfro_laterdm_diff.append(diffevks)
                alpha_medfro_laterdm_diff = alpha_medfro_laterdm_diff[0]
            elif cond == 4:
                diffevks = np.subtract(alpha_medfro_EarlyMir[0], alpha_medfro_EarlyLateAligned[0])
                alpha_medfro_earlymir_diff.append(diffevks)
                alpha_medfro_earlymir_diff = alpha_medfro_earlymir_diff[0]
            elif cond == 5:
                diffevks = np.subtract(alpha_medfro_LateMir[0], alpha_medfro_EarlyLateAligned[0])
                alpha_medfro_latemir_diff.append(diffevks)
                alpha_medfro_latemir_diff = alpha_medfro_latemir_diff[0]
    elif band == 1:
        for cond in range(0, len(diffconds)):
            if cond == 0:
                diffevks = np.subtract(beta_medfro_EarlyRot[0], beta_medfro_EarlyLateAligned[0])
                beta_medfro_earlyrot_diff.append(diffevks)
                beta_medfro_earlyrot_diff = beta_medfro_earlyrot_diff[0] #to keep shape of object consistent
            elif cond == 1:
                diffevks = np.subtract(beta_medfro_LateRot[0], beta_medfro_EarlyLateAligned[0])
                beta_medfro_laterot_diff.append(diffevks)
                beta_medfro_laterot_diff = beta_medfro_laterot_diff[0]
            elif cond == 2:
                diffevks = np.subtract(beta_medfro_EarlyRdm[0], beta_medfro_EarlyLateAligned[0])
                beta_medfro_earlyrdm_diff.append(diffevks)
                beta_medfro_earlyrdm_diff = beta_medfro_earlyrdm_diff[0]
            elif cond == 3:
                diffevks = np.subtract(beta_medfro_LateRdm[0], beta_medfro_EarlyLateAligned[0])
                beta_medfro_laterdm_diff.append(diffevks)
                beta_medfro_laterdm_diff = beta_medfro_laterdm_diff[0]
            elif cond == 4:
                diffevks = np.subtract(beta_medfro_EarlyMir[0], beta_medfro_EarlyLateAligned[0])
                beta_medfro_earlymir_diff.append(diffevks)
                beta_medfro_earlymir_diff = beta_medfro_earlymir_diff[0]
            elif cond == 5:
                diffevks = np.subtract(beta_medfro_LateMir[0], beta_medfro_EarlyLateAligned[0])
                beta_medfro_latemir_diff.append(diffevks)
                beta_medfro_latemir_diff = beta_medfro_latemir_diff[0]

In [13]:
# save aligned (baseline) subtracted tfr data across participants in each condition (used for plotting signals)
alpha_flists = [alpha_medfro_earlyrot_diff, alpha_medfro_laterot_diff, 
                alpha_medfro_earlyrdm_diff, alpha_medfro_laterdm_diff, 
                alpha_medfro_earlymir_diff, alpha_medfro_latemir_diff]
beta_flists = [beta_medfro_earlyrot_diff, beta_medfro_laterot_diff, 
               beta_medfro_earlyrdm_diff, beta_medfro_laterdm_diff, 
               beta_medfro_earlymir_diff, beta_medfro_latemir_diff]

alpha_conditionnames = ['alpha_earlyrot', 'alpha_laterot', 'alpha_earlyrdm', 'alpha_laterdm', 'alpha_earlymir', 'alpha_latemir']
beta_conditionnames = ['beta_earlyrot', 'beta_laterot', 'beta_earlyrdm', 'beta_laterdm', 'beta_earlymir', 'beta_latemir']

freq_bands = ['alpha', 'beta']

for band in range(0, len(freq_bands)):
    if band == 0:
        for f in range(0, len(alpha_flists)):
            df = pd.DataFrame(alpha_flists[f]).T #no need to index [0]
            df['time'] = time

            filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_medfro_DiffWaves_%s_%s.csv' % (alpha_conditionnames[f], erps))
            df.to_csv(filename)
    elif band == 1:
        for f in range(0, len(beta_flists)):
            df = pd.DataFrame(beta_flists[f]).T
            df['time'] = time

            filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_medfro_DiffWaves_%s_%s.csv' % (beta_conditionnames[f], erps))
            df.to_csv(filename)

In [14]:
# Compare EARLY VS LATE for each perturbation
# Generate a data frame to tabulate condition, cluster indices, cluster timepts, p values
# This information can then be included in plots
p = 0.05
perms = 1000

condition = []
clust_idx_start = []
clust_idx_end = []
time_start = []
time_end = []
p_values = []

freq_bands = ['alpha', 'beta']

for band in range(0, len(freq_bands)):
    if band == 0:
        alpha_conditionnames = ['alpha_medfro_rot', 'alpha_medfro_rdm', 'alpha_medfro_mir']
        for c in range(0, len(alpha_conditionnames)):
            if c == 0:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(alpha_medfro_laterot_diff, alpha_medfro_earlyrot_diff, p, perms)
        #         print(clust_idx, clust_pvals)
                if len(clust_idx) == 0:
                    condition.append(alpha_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(alpha_conditionnames[c])
            
            elif c == 1:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(alpha_medfro_laterdm_diff, alpha_medfro_earlyrdm_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(alpha_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(alpha_conditionnames[c])
            
            elif c == 2:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(alpha_medfro_latemir_diff, alpha_medfro_earlymir_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(alpha_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(alpha_conditionnames[c])
            
    elif band == 1:
        beta_conditionnames = ['beta_medfro_rot', 'beta_medfro_rdm', 'beta_medfro_mir']
        for c in range(0, len(beta_conditionnames)):
            if c == 0:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(beta_medfro_laterot_diff, beta_medfro_earlyrot_diff, p, perms)
        #         print(clust_idx, clust_pvals)
                if len(clust_idx) == 0:
                    condition.append(beta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(beta_conditionnames[c])
            
            elif c == 1:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(beta_medfro_laterdm_diff, beta_medfro_earlyrdm_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(beta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(beta_conditionnames[c])
            
            elif c == 2:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(beta_medfro_latemir_diff, beta_medfro_earlymir_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(beta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(beta_conditionnames[c])
        
perm_test = pd.DataFrame(
    {'condition': condition,
     'clust_idx_start': clust_idx_start,
     'clust_idx_end': clust_idx_end,
     'time_start': time_start,
     'time_end': time_end,
     'p_values': p_values})

perm_test_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_Permutation_test_EarlyvsLate_%s_%s.csv' % (erps, roi))
perm_test.to_csv(perm_test_filename)

stat_fun(H1): min=-0.641803 max=1.667319
Running initial clustering …
Found 0 clusters
stat_fun(H1): min=-3.093379 max=2.629219
Running initial clustering …
Found 4 clusters


C:\Users\Raphael\AppData\Local\Temp\ipykernel_2160\3082407755.py:11: RuntimeWarning: No clusters found, returning empty H0, clusters, and cluster_pv
  T_0, clust_idx, clust_pvals, H0 = mne.stats.permutation_cluster_1samp_test(data, threshold = thresh,


  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]

stat_fun(H1): min=-1.009411 max=1.155527
Running initial clustering …
Found 0 clusters
stat_fun(H1): min=-2.769811 max=2.083327
Running initial clustering …
Found 2 clusters


C:\Users\Raphael\AppData\Local\Temp\ipykernel_2160\3082407755.py:11: RuntimeWarning: No clusters found, returning empty H0, clusters, and cluster_pv
  T_0, clust_idx, clust_pvals, H0 = mne.stats.permutation_cluster_1samp_test(data, threshold = thresh,


  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]

stat_fun(H1): min=-2.654840 max=1.877930
Running initial clustering …
Found 2 clusters


  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]

stat_fun(H1): min=-1.219008 max=0.787919
Running initial clustering …
Found 0 clusters


C:\Users\Raphael\AppData\Local\Temp\ipykernel_2160\3082407755.py:11: RuntimeWarning: No clusters found, returning empty H0, clusters, and cluster_pv
  T_0, clust_idx, clust_pvals, H0 = mne.stats.permutation_cluster_1samp_test(data, threshold = thresh,


In [15]:
# Next step is to subtract early from late condition, to generate a single signal for each perturbation
diffconds = ['rot', 'rdm', 'mir']
freq_bands = ['alpha', 'beta']

alpha_medfro_rot_diff = []
alpha_medfro_rdm_diff = []
alpha_medfro_mir_diff = []

beta_medfro_rot_diff = []
beta_medfro_rdm_diff = []
beta_medfro_mir_diff = []

for band in range(0, len(freq_bands)):
    if band == 0:
        for cond in range(0, len(diffconds)):
            if cond == 0:
                diffevks = np.subtract(alpha_medfro_laterot_diff, alpha_medfro_earlyrot_diff)
                alpha_medfro_rot_diff.append(diffevks)
                alpha_medfro_rot_diff = alpha_medfro_rot_diff[0] #to keep shape of object consistent
            elif cond == 1:
                diffevks = np.subtract(alpha_medfro_laterdm_diff, alpha_medfro_earlyrdm_diff)
                alpha_medfro_rdm_diff.append(diffevks)
                alpha_medfro_rdm_diff = alpha_medfro_rdm_diff[0]
            elif cond == 2:
                diffevks = np.subtract(alpha_medfro_latemir_diff, alpha_medfro_earlymir_diff)
                alpha_medfro_mir_diff.append(diffevks)
                alpha_medfro_mir_diff = alpha_medfro_mir_diff[0]
    elif band == 1:
        for cond in range(0, len(diffconds)):
            if cond == 0:
                diffevks = np.subtract(beta_medfro_laterot_diff, beta_medfro_earlyrot_diff)
                beta_medfro_rot_diff.append(diffevks)
                beta_medfro_rot_diff = beta_medfro_rot_diff[0] #to keep shape of object consistent
            elif cond == 1:
                diffevks = np.subtract(beta_medfro_laterdm_diff, beta_medfro_earlyrdm_diff)
                beta_medfro_rdm_diff.append(diffevks)
                beta_medfro_rdm_diff = beta_medfro_rdm_diff[0]
            elif cond == 2:
                diffevks = np.subtract(beta_medfro_latemir_diff, beta_medfro_earlymir_diff)
                beta_medfro_mir_diff.append(diffevks)
                beta_medfro_mir_diff = beta_medfro_mir_diff[0]

In [16]:
# save early and late subtracted tfr data across participants in each condition (used for plotting signals)
alpha_flists = [alpha_medfro_rot_diff, alpha_medfro_rdm_diff, alpha_medfro_mir_diff]
beta_flists = [beta_medfro_rot_diff, beta_medfro_rdm_diff, beta_medfro_mir_diff]

alpha_conditionnames = ['alpha_rot', 'alpha_rdm', 'alpha_mir']
beta_conditionnames = ['beta_rot', 'beta_rdm', 'beta_mir']

freq_bands = ['alpha', 'beta']

for band in range(0, len(freq_bands)):
    if band == 0:
        for f in range(0, len(alpha_flists)):
            df = pd.DataFrame(alpha_flists[f]).T #no need to index [0]
            df['time'] = time

            filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_medfro_DiffWaves_EvL_%s_%s.csv' % (alpha_conditionnames[f], erps))
            df.to_csv(filename)
    elif band == 1:
        for f in range(0, len(beta_flists)):
            df = pd.DataFrame(beta_flists[f]).T
            df['time'] = time

            filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_medfro_DiffWaves_EvL_%s_%s.csv' % (beta_conditionnames[f], erps))
            df.to_csv(filename)

In [17]:
# Compare EARLY VS LATE for each perturbation
# Generate a data frame to tabulate condition, cluster indices, cluster timepts, p values
# This information can then be included in plots
p = 0.05
perms = 1000

condition = []
clust_idx_start = []
clust_idx_end = []
time_start = []
time_end = []
p_values = []

freq_bands = ['alpha', 'beta']

for band in range(0, len(freq_bands)):
    if band == 0:
        alpha_conditionnames = ['alpha_medfro_rotvmir', 'alpha_medfro_rotvrdm', 'alpha_medfro_mirvrdm']
        for c in range(0, len(alpha_conditionnames)):
            if c == 0:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(alpha_medfro_rot_diff, alpha_medfro_mir_diff, p, perms)
        #         print(clust_idx, clust_pvals)
                if len(clust_idx) == 0:
                    condition.append(alpha_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(alpha_conditionnames[c])
            
            elif c == 1:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(alpha_medfro_rot_diff, alpha_medfro_rdm_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(alpha_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(alpha_conditionnames[c])
            
            elif c == 2:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(alpha_medfro_mir_diff, alpha_medfro_rdm_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(alpha_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(alpha_conditionnames[c])
            
    elif band == 1:
        beta_conditionnames = ['beta_medfro_rotvmir', 'beta_medfro_rotvrdm', 'beta_medfro_mirvrdm']
        for c in range(0, len(beta_conditionnames)):
            if c == 0:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(beta_medfro_rot_diff, beta_medfro_mir_diff, p, perms)
        #         print(clust_idx, clust_pvals)
                if len(clust_idx) == 0:
                    condition.append(beta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(beta_conditionnames[c])
            
            elif c == 1:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(beta_medfro_rot_diff, beta_medfro_rdm_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(beta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(beta_conditionnames[c])
            
            elif c == 2:
                T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(beta_medfro_mir_diff, beta_medfro_rdm_diff, p, perms)
                if len(clust_idx) == 0:
                    condition.append(beta_conditionnames[c])
                    clust_idx_start.append(np.nan)
                    clust_idx_end.append(np.nan)
                    time_start.append(np.nan)
                    time_end.append(np.nan)
                    p_values.append(np.nan)
                else:    
                    for clust in range(0, len(clust_idx)):
                        cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                        cluster_start = cluster.start
                        clust_idx_start.append(cluster_start)
        
                        cluster_end = cluster.stop
                        clust_idx_end.append(cluster_end)
        
                        time_idx_start = time[cluster_start]
                        time_start.append(time_idx_start)
        
                        time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                        time_end.append(time_idx_end)
        
                        clust_p = clust_pvals[clust]
                        p_values.append(clust_p)
        
                        condition.append(beta_conditionnames[c])
        
perm_test = pd.DataFrame(
    {'condition': condition,
     'clust_idx_start': clust_idx_start,
     'clust_idx_end': clust_idx_end,
     'time_start': time_start,
     'time_end': time_end,
     'p_values': p_values})

perm_test_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_Permutation_test_PerturbTypeComp_%s_%s.csv' % (erps, roi))
perm_test.to_csv(perm_test_filename)

stat_fun(H1): min=-0.501438 max=1.168611
Running initial clustering …
Found 0 clusters
stat_fun(H1): min=-1.516902 max=1.960875
Running initial clustering …
Found 0 clusters
stat_fun(H1): min=-1.800531 max=1.544852
Running initial clustering …
Found 0 clusters
stat_fun(H1): min=-1.228202 max=1.950245
Running initial clustering …
Found 0 clusters
stat_fun(H1): min=-2.553462 max=2.746528
Running initial clustering …
Found 5 clusters


C:\Users\Raphael\AppData\Local\Temp\ipykernel_2160\3082407755.py:11: RuntimeWarning: No clusters found, returning empty H0, clusters, and cluster_pv
  T_0, clust_idx, clust_pvals, H0 = mne.stats.permutation_cluster_1samp_test(data, threshold = thresh,
C:\Users\Raphael\AppData\Local\Temp\ipykernel_2160\3082407755.py:11: RuntimeWarning: No clusters found, returning empty H0, clusters, and cluster_pv
  T_0, clust_idx, clust_pvals, H0 = mne.stats.permutation_cluster_1samp_test(data, threshold = thresh,
C:\Users\Raphael\AppData\Local\Temp\ipykernel_2160\3082407755.py:11: RuntimeWarning: No clusters found, returning empty H0, clusters, and cluster_pv
  T_0, clust_idx, clust_pvals, H0 = mne.stats.permutation_cluster_1samp_test(data, threshold = thresh,
C:\Users\Raphael\AppData\Local\Temp\ipykernel_2160\3082407755.py:11: RuntimeWarning: No clusters found, returning empty H0, clusters, and cluster_pv
  T_0, clust_idx, clust_pvals, H0 = mne.stats.permutation_cluster_1samp_test(data, threshold = 

  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]

stat_fun(H1): min=-1.786983 max=2.226359
Running initial clustering …
Found 1 cluster


  0%|          | Permuting : 0/999 [00:00<?,       ?it/s]